# Milestone Project

The goal is to simulate the quantum adiabatic algorithm solving a small instance of maximum independence set on the graph defined by the adjacency matrix $M$, where

$$ 
M = \begin{pmatrix}
0 & 1 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 & 1 \\
0 & 0 & 0 & 1 & 1 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 
\end{pmatrix}  
$$

### Method
1. Construct $H_{\mathrm{Ising}}$ for this graph
2. Verify that the lowest energy state is indeed the maximum independent set $|10011\rangle$, which is represented by a column vector of length $2^5$ with a one in the 19<sup>th</sup> position, and zeros elsewhere
3. Simulate an adiabatic algorithm solving the problem with a time dependent Hamiltonian 


#### Import necessary libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt

#### Helper functions

In [44]:
pauli_matrices = {
    'x':np.matrix(([0, 1], [1, 0]), dtype=complex),
    'y':np.matrix(([0, -1j], [1j, 0]), dtype=complex),
    'z':np.matrix(([1, 0], [0, -1]), dtype=complex)
}

def generate_sigma_j(j: int, axis: str, n:int) -> np.matrix:
    """
    Inputs:
    j: the qubit being operated on
    axis: x, y or z 
    n: number of qubits in the system
    Returns:
    Matrix representation of the axis pauli operation
    on the jth qubit
    """
    # Set pauli matrix according to input axis
    sigma = pauli_matrices[axis]
    
    # Initialise sigma_j as 1
    sigma_j = 1

    # Iterate through to n
    for i in range(1, n + 1):
        if i == j:
            sigma_j = np.kron(sigma_j, sigma)
        else:
            sigma_j = np.kron(sigma_j, np.identity(2))
    
    return sigma_j

Test helper functions

In [29]:
# Write helper function tests

#### 1. Construct $H_{\mathrm{Ising}}$
$$
H_{\mathrm{Ising}} = \sum_{k = 1}^n \sum_{j = k + 1} ^ n J_{kj} \sigma_k^z \sigma_j^z + \sum_{j=1}^n h_j \sigma_j^z,
$$
where $J = M$, and
$$
h_k = -\sum_{j=1}^n(M_{kj} + M_{jk}) + \kappa
$$

In [84]:
def generate_strengths(M: np.matrix, kappa:float) -> tuple[np.matrix, np.ndarray]:
    """
    Inputs are M the adjacancy matrix and kappa the variable that rewards
    more independence
    Returns J and h, the coupling and field strengths respectively
    """
    # Set n to be the number of qubits in the system
    n = M.shape[0]

    # Initialise h as an array of kappas
    h = np.ones((n), dtype=complex) * kappa

    # Generate h - TODO method could probably be vectorised
    for k in range(n):
        for j in range(n):
            h[k] += -(M[k, j] + M[j, k])
    
    return M, h

def construct_H_ising(J: np.ndarray, h: np.ndarray) -> np.matrix:
    """
    Inputs: J and h are the coupling and field strengths respectively
    Returns: The Ising Hamiltonian, a 2^n x 2^n matrix
    """
    n = h.size
    H_ising = np.zeros((2**n, 2**n), dtype=complex)

    # Add the first sum
    for k in range(n):
        for j in range(k + 1, n):
            sigma_j = generate_sigma_j(j, 'z', n)
            sigma_k = generate_sigma_j(k, 'z', n)
            H_ising += J[k, j] * sigma_k * sigma_j
    
    # Add the second sum
    for j in range(n):
        sigma_j = generate_sigma_j(j, 'z', n)
        H_ising += h[j] * sigma_j

    return H_ising

Test these functions with the example from the project book.

For the three qubit problem defined by the graph represented by the adjacency matrix

$$
M = \begin{pmatrix}
0 & 1 & 0 \\
0 & 0 & 1 \\
0 & 0 & 0 
\end{pmatrix},
$$

We expect the problem Hamiltonian to take the form below.

In [88]:
kappa = 0.5
M = np.matrix([
    [0, 1, 0],
    [0, 0, 1],
    [0, 0, 0]
])
expected_H = np.diag(np.array([
    -2 + 3*kappa, 
    -2 + kappa,
    -2 + kappa, 
    2 - kappa,
    -2 + kappa,
    -2 - kappa,
    2 - kappa,
    6 - 3*kappa
], dtype=complex))

J, h = generate_strengths(M, kappa)

if np.array_equal(h, np.array([-0.5, -1.5, -0.5])):
    print("Test 1 passed")
else:
    print(f"Test 1 failed, expected\n {np.array([-0.5, -1.5, -0.5])},\n got {h}")

my_H = construct_H_ising(J, h)

if np.array_equal(my_H, expected_H):
    print("Test Passed!")
else:
    print(f"Test failed, expected \n {expected_H},\n got \n {my_H}.")

Test 1 passed
Test failed, expected 
 [[-0.5+0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j -1.5+0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  0. +0.j -1.5+0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  0. +0.j  0. +0.j  1.5+0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  0. +0.j  0. +0.j  0. +0.j -1.5+0.j  0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j -2.5+0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  1.5+0.j  0. +0.j]
 [ 0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  4.5+0.j]],
 got 
 [[-0.5+0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  0. +0.j -1.5+0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  0. +0.j  0. +0.j -1.5+0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  0. +0.j  0. +0.j  0. +0.j -1.5+0.j  0. +0